In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("F:\\Real_Combine.csv")
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [4]:
df=df.dropna()

In [5]:
x=df.iloc[:,:-1]
x.shape

(1092, 8)

In [6]:
y=df.iloc[:,-1]
y.shape

(1092,)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=51)
print("shape of x_train : ",x_train.shape)
print("shape of x_test : ",x_test.shape)
print("shape of y_train : ",y_train.shape)
print("shape of y_test : ",y_test.shape)

shape of x_train :  (764, 8)
shape of x_test :  (328, 8)
shape of y_train :  (764,)
shape of y_test :  (328,)


#  Feature Scaling

In [9]:
from sklearn.preprocessing import  StandardScaler

In [10]:
sc=StandardScaler()
sc.fit(x_train,y_train)
sc.transform(x_train)
x_train_sc=sc.transform(x_train)
x_test_sc=sc.transform(x_test)

# MODEL SELECTION

# Linear Regression

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
lr= LinearRegression()

In [13]:
lr.fit(x_train,y_train)

LinearRegression()

In [14]:
lr.score(x_test,y_test)

0.5206956053582055

In [15]:
lr.fit(x_train_sc,y_train)

LinearRegression()

In [16]:
lr.score(x_test_sc,y_test)

0.5206956053582055

In [17]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [18]:
rd=Ridge()

In [19]:
rd.fit(x_train_sc,y_train)

Ridge()

In [20]:
rd.score(x_test_sc,y_test)

0.520831924298794

#  support vector machine

In [21]:
from sklearn.svm import SVR

In [22]:
svc=SVR()

In [23]:
svc.fit(x_train_sc,y_train)
svc.score(x_test_sc,y_test)

0.42631203159922615

In [24]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [25]:
y_pred_svcr=svc.predict(x_test_sc)
mse=mean_squared_error(y_test,y_pred_svcr)
rmse=np.sqrt(mse)
print(mse)
print(rmse)

4107.103163213319
64.08668475754787


#  Random Forest Regression

In [26]:
from sklearn.ensemble import RandomForestRegressor

In [27]:
rfr=RandomForestRegressor()
rfr.fit(x_train,y_train)
rfr.score(x_test,y_test)

0.7776211941071478

In [28]:
y_pred_rfr=rfr.predict(x_test)
mse=mean_squared_error(y_test,y_pred_rfr)
rmse=np.sqrt(mse)
print(mse)
print(rmse)

1592.03739213873
39.90034325840731


# Decision Tree Regression

In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
dtc=DecisionTreeRegressor()
dtc.fit(x_train,y_train)

DecisionTreeRegressor()

In [31]:
dtc.score(x_test,y_test)

0.5757725671339091

In [32]:
y_pred_dtc=dtc.predict(x_test)
mse=mean_squared_error(y_test,y_pred_dtc)
rmse=np.sqrt(mse)
print(mse)
print(rmse)

3037.096692655403
55.109860212628035


#  XGBoost 

In [33]:
import xgboost as XGBoost

In [34]:
from xgboost import XGBRegressor
xgb=XGBRegressor()
xgb.fit(x_train_sc,y_train)
xgb.score(x_test_sc,y_test)

[20:47:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.7495492635241426

In [35]:
y_pred_xgb=xgb.predict(x_test_sc)
mse=mean_squared_error(y_test,y_pred_xgb)
rmse=np.sqrt(mse)
print(mse)

1793.007817257392


# Hyper parameter Tuning using Randomized search

In [36]:
from sklearn.model_selection import RandomizedSearchCV

In [61]:
params={
    "n_estimators":[100,200,300,400,500],
    "max_depth":[4,6,8,10,12],
    "min_samples_split":[2,4,6,8],
    "min_samples_leaf":[1,2,3,4],
    "min_weight_fraction_leaf":[0.0,0.1,0.3,0.4,0.5],
    "max_features":[ 'sqrt','log2',"auto"],
    "criterion":["mse","entropy"]
}


In [62]:
rscv=RandomizedSearchCV(rfr,param_distributions=params,n_jobs=-1,verbose=True)
rscv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   21.0s finished


RandomizedSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'criterion': ['mse', 'entropy'],
                                        'max_depth': [4, 6, 8, 10, 12],
                                        'max_features': ['sqrt', 'log2',
                                                         'auto'],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [2, 4, 6, 8],
                                        'min_weight_fraction_leaf': [0.0, 0.1,
                                                                     0.3, 0.4,
                                                                     0.5],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   verbose=True)

In [63]:
rscv.best_params_

{'n_estimators': 400,
 'min_weight_fraction_leaf': 0.0,
 'min_samples_split': 4,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 10,
 'criterion': 'mse'}

In [65]:
rscv.best_estimator_

RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=2,
                      min_samples_split=4, n_estimators=400)

In [66]:
rdf=RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=2,
                      min_samples_split=4, n_estimators=400)
rdf.fit(x_train,y_train)

RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=2,
                      min_samples_split=4, n_estimators=400)

In [67]:
rdf.score(x_test,y_test)

0.770743223086072

In [68]:
y_pred=rdf.predict(x_test)

In [69]:
mse=mean_squared_error(y_pred,y_test)
print(mse)

1641.2776378700396


In [70]:
rmse=np.sqrt(mse)

In [71]:
print(rmse)

40.51268490078187


# save model

In [79]:
import joblib

In [80]:
joblib.dump(rdf,"model.pkl")

['model.pkl']

In [81]:
model=joblib.load("model.pkl")